# Analisando os dados disponibilizados

### 1) Importar bibliotecas

In [290]:
import pandas as pd
from unidecode import unidecode

### 2) Ler dados brutos

In [291]:
escolas = pd.read_csv("dados_brutos/escolas.csv")

material_didatico = pd.read_csv("dados_brutos/material_didatico.csv")

subprefeituras = pd.read_csv("dados_brutos/subprefeituras.csv")

### 3) Criar cópias dos DFs

In [292]:
subprefeituras_copia = subprefeituras.copy()

In [293]:
material_didatico_copia = material_didatico.copy()

In [294]:
escolas_copia = escolas.copy()

### 4) Tratamento dos DFs

#### 4.1) Tratamento geral

1) aplicar lower para todas as colunas
2) nome das colunas em snake_case
3) strings não devem conter acentos
4) todas as strings devem estar em maiúsculo

In [295]:
def renomear_colunas(df):
    new_columns = {col: unidecode(col.lower().strip()) for col in df.columns}
    df.rename(columns=new_columns, inplace=True)
    for col in df.columns:
        df[col] = df[col].apply(lambda x: unidecode(str(x).upper())) if df[col].dtypes == 'object' else df[col]

In [296]:
dfs = [escolas_copia, material_didatico_copia, subprefeituras_copia]

for df in dfs:
    renomear_colunas(df)
    display(df.head(5))

,id,escolas_postos,bairro,endereco,lat,lon
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/No,"-22,880888","-43,225326"
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,"-22,889574","-43,236202"
2,483,EM CELESTINO SILVA,CENTRO,"R. DO LAVRADIO, 56","-22,909293","-43,183579"
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,"-22,897629","-43,227456"
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,"-22,924412","-43,208579"


,id,quantidade
0,178,20
1,634,121
2,483,220
3,476,190
4,132,67


,nome,subprefeitura
0,SANTO CRISTO,CENTRO
1,GAMBOA,CENTRO
2,SAUDE,CENTRO
3,CAJU,CENTRO
4,CENTRO,CENTRO


### 4.2) Tratamento para o DF material didático

#### O que deve ser feito:

1. Converter tipo coluna de objeto para inteiro

2. Substituir NaN e None por 0

3. Substituir '' e ' ' por 0

4. Substituir 'NaN' por 0

5. Converter valor numérico do tipo objeto para int (Ex.: '356' para 356)

Remover valores nulos


In [297]:
material_didatico_copia['quantidade'].fillna('0', inplace=True)

Verifica valores diferentes de números

In [298]:
for i, row in material_didatico_copia.iterrows():
    qnt = row['quantidade']
    if not qnt.isdigit():
        print(qnt)

''
'NAN'
NAN
'397'
'NAN'
'395'
NAN
NAN
' '
NAN


Converte todos os valores da coluna para int (Modo 1)

In [299]:
material_didatico_copia['quantidade'] = material_didatico_copia['quantidade'].apply(lambda x: int(x) if x.isdigit() else 0)

In [300]:
# # Modo 2
# import re

# def normaliza_quantidade(valor) -> int:
#     if valor:
#         normaliza_valor = re.sub(r"[^\d: ]", "", valor.strip())
#         if normaliza_valor:
#             return int(normaliza_valor)
#         else:
#             return 0
#     else:
#         return 0
    
# material_didatico['quantidade'] = material_didatico['quantidade'].apply(lambda x: normaliza_quantidade(x))

Filtra dataframe com quantidade de material diferente de zero

In [301]:
material_didatico_copia = material_didatico_copia[material_didatico_copia['quantidade'] != 0]

Verifica existência de valores nulos

In [302]:
material_didatico_copia.isna().sum()

id            0
quantidade    0
dtype: int64

### 4.3) Tratamento para o DF subprefeituras

Verifica existência de valores duplicados

In [303]:
subprefeituras_copia.duplicated().sum()

0

### 4.4) Tratamento para o DF escola

Normaliza coluna lat com 5 casas decimais

In [304]:
for col in ['lat', 'lon']:
    escolas_copia[col] = escolas_copia[col].str.replace(",", ".").astype(float).round(5)

##### Substitui abreviações de endereço

In [305]:
# verifica possíveis abreviações
abreviacoes = set()

for i, linha_escola in escolas_copia.iterrows():
    lista_palavra_endereco = linha_escola['endereco'].split()
    for palavra in lista_palavra_endereco:
        if len(palavra) <= 3 and not palavra.isdigit():
            abreviacoes.update([palavra])

abreviacoes

{'ANA',
 'AV',
 'AV.',
 'DA',
 'DAS',
 'DE',
 'DO',
 'DOM',
 'DOS',
 'E',
 'JR',
 'KM',
 'PAU',
 'R.',
 'RIO',
 'RUA',
 'S/N',
 'SD.',
 'UBA'}

In [306]:
# verifica possíveis abreviações para logradouros
filtro_abreviacoes = [abreviacao for abreviacao in abreviacoes if abreviacao.startswith('R') or abreviacao.startswith('AV')]
filtro_abreviacoes

['RUA', 'AV', 'R.', 'RIO', 'AV.']

In [307]:
escolas_copia['endereco'] = escolas_copia['endereco'].str.replace('R. ', 'RUA ', regex=True).replace('AV. ', 'AVENIDA ', regex=True).replace('AV ', 'AVENIDA ', regex=True)

In [308]:
escolas_copia

,id,escolas_postos,bairro,endereco,lat,lon
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/No,-22.88089,-43.22533
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,-22.88957,-43.23620
2,483,EM CELESTINO SILVA,CENTRO,"RUA DO LAVRADIO, 56",-22.90929,-43.18358
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,-22.89763,-43.22746
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,-22.92441,-43.20858
...,...,...,...,...,...,...
147,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,ESTRADA DO MAGARCA 9.183,-22.98046,-43.64354
148,301,EM JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO, S/No",-22.95316,-43.57741
149,215,E.M. NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO S/N.o,-23.00908,-43.53758
150,606,EM PROFESSOR CASTILHO,ILHA DE GUARATIBA,CAMINHO DA MATRIZ 4406,-22.99412,-43.59368


Valida normalização de abreviações do campo endereco

In [309]:
for i, linha_escola_2 in escolas_copia.iterrows():
    if linha_escola_2['endereco'].startswith('R. ')  or \
       linha_escola_2['endereco'].startswith('AV. ') or \
       linha_escola_2['endereco'].startswith('AV '):
        print(linha_escola_2)

Adiciona vírgula para o número do endereço e unifica os SN

In [310]:
# verifica possíveis abreviações
numeros = list()
enderecos_sem_numeros = list()

numeros = [linha_escola['endereco'].split()[-1] for i, linha_escola in escolas_copia.iterrows()]

for item in numeros:
    if not item.isdigit():
        enderecos_sem_numeros.append(item)

set_enderecos_sem_numeros = set(enderecos_sem_numeros)
print(set_enderecos_sem_numeros)

{'S/N.deg', 'MENDANHA', 'VITORIA', '23.364', '483-639', '1.565', '4.035', 'CESARAO', '9.183', 'S/No', 'S/Ndeg', 'URUCANIA', 'CORREA', 'S/N', 'IGARAPE-ACU', 'NERI', 'S/N.o'}


In [311]:
# utiliza apenas os sem números
for item in set_enderecos_sem_numeros:
    if item.startswith('S'):
        print(item)

S/N.deg
S/No
S/Ndeg
S/N
S/N.o


In [312]:
import re
# Função para adicionar vírgula entre endereço e número
def add_comma_to_address(address):
    # Use expressões regulares para encontrar o número no final do endereço
    match = re.search(r'(\d+(\.\d+)?)$', address)
    
    if match:
        # Se um número for encontrado, adicione uma vírgula antes dele
        address_with_comma = re.sub(r'(\d+(\.\d+)?)$', r', \1', address)
        return address_with_comma
    else:
        return address

# Função para tratar as diversas maneiras de endereços sem numero para `SN``
def replace_s_n(text):
    # Lista de trechos a serem substituídos
    to_replace = ["S/N.deg", "S/No", "S/Ndeg", "S/N", "S/N.o", "SN.o"]
    # to_replace = ["S/No", "S/N.o", "S/N", "S/N.deg", "S/Ndeg", "SNdeg", "SN.deg"]
    
    # Substituir os trechos na string
    for item in to_replace:
        text = text.replace(item, ",SN")
    
    return text

In [313]:
escolas_copia['endereco'] = escolas_copia['endereco'].apply(add_comma_to_address)
escolas_copia

,id,escolas_postos,bairro,endereco,lat,lon
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/No,-22.88089,-43.22533
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,"RUA EBANO , 187",-22.88957,-43.23620
2,483,EM CELESTINO SILVA,CENTRO,"RUA DO LAVRADIO, , 56",-22.90929,-43.18358
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,"PRACA ARGENTINA , 20",-22.89763,-43.22746
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,"PRACA CONDESSA PAULO DE FRONTIN , 45",-22.92441,-43.20858
...,...,...,...,...,...,...
147,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,"ESTRADA DO MAGARCA , 9.183",-22.98046,-43.64354
148,301,EM JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO, S/No",-22.95316,-43.57741
149,215,E.M. NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO S/N.o,-23.00908,-43.53758
150,606,EM PROFESSOR CASTILHO,ILHA DE GUARATIBA,"CAMINHO DA MATRIZ , 4406",-22.99412,-43.59368


In [314]:
escolas_copia['endereco'] = escolas_copia['endereco'].apply(replace_s_n)
escolas_copia

,id,escolas_postos,bairro,endereco,lat,lon
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,"RUA CARLOS SEIDL ,SN",-22.88089,-43.22533
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,"RUA EBANO , 187",-22.88957,-43.23620
2,483,EM CELESTINO SILVA,CENTRO,"RUA DO LAVRADIO, , 56",-22.90929,-43.18358
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,"PRACA ARGENTINA , 20",-22.89763,-43.22746
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,"PRACA CONDESSA PAULO DE FRONTIN , 45",-22.92441,-43.20858
...,...,...,...,...,...,...
147,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,"ESTRADA DO MAGARCA , 9.183",-22.98046,-43.64354
148,301,EM JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO, ,SN",-22.95316,-43.57741
149,215,E.M. NARCISA AMALIA,ILHA DE GUARATIBA,"ESTRADA TEODORETO DE CAMARGO ,,SN",-23.00908,-43.53758
150,606,EM PROFESSOR CASTILHO,ILHA DE GUARATIBA,"CAMINHO DA MATRIZ , 4406",-22.99412,-43.59368


In [315]:
endereco_series = escolas_copia['endereco']

condicao = endereco_series.str.count(',') > 1


endereco_series.loc[condicao] = endereco_series.loc[condicao].str.replace(', ', ',')
escolas_copia['endereco'] = endereco_series

endereco_series.loc[condicao] = endereco_series.loc[condicao].str.replace(',,', ',')
escolas_copia['endereco'] = endereco_series


endereco_series.loc[condicao] = endereco_series.loc[condicao].str.replace(' ,', ',')
escolas_copia['endereco'] = endereco_series

C:\Users\wpett\AppData\Local\Temp\ipykernel_6192\466586041.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  endereco_series.loc[condicao] = endereco_series.loc[condicao].str.replace(', ', ',')
C:\Users\wpett\AppData\Local\Temp\ipykernel_6192\466586041.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  endereco_series.loc[condicao] = endereco_series.loc[condicao].str.replace(',,', ',')
C:\Users\wpett\AppData\Local\Temp\ipykernel_6192\466586041.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [321]:
escolas_copia

,id,escolas_postos,bairro,endereco,lat,lon
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,"RUA CARLOS SEIDL ,SN",-22.88089,-43.22533
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,"RUA EBANO , 187",-22.88957,-43.23620
2,483,EM CELESTINO SILVA,CENTRO,"RUA DO LAVRADIO,56",-22.90929,-43.18358
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,"PRACA ARGENTINA , 20",-22.89763,-43.22746
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,"PRACA CONDESSA PAULO DE FRONTIN , 45",-22.92441,-43.20858
...,...,...,...,...,...,...
147,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,"ESTRADA DO MAGARCA , 9.183",-22.98046,-43.64354
148,301,EM JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO,SN",-22.95316,-43.57741
149,215,E.M. NARCISA AMALIA,ILHA DE GUARATIBA,"ESTRADA TEODORETO DE CAMARGO,SN",-23.00908,-43.53758
150,606,EM PROFESSOR CASTILHO,ILHA DE GUARATIBA,"CAMINHO DA MATRIZ , 4406",-22.99412,-43.59368
